# Pyresample
> Pyresample is a Python package for resampling (reprojection) of earth observing satellite data. Pyresample handles both resampling of gridded data (e.g. geostationary satellites) and swath data (polar orbiting satellites). Pyresample can use multiple processor cores for resampling. Pyresample supports masked arrays.

[Documentation](https://pyresample.readthedocs.io/en/latest/)


## Let's get the data
If you have not got MODIS data, go to SatPy notebook
We will use satpy to load the data
but we won't use it here for anything else

In [ ]:
from satpy import Scene
import glob

filenames = glob.glob('./M*238.1435')
scn = Scene(reader='hdfeos_l1b',
            filenames=filenames)

scn.load(['1',
          'latitude',
          'longitude'])

In [ ]:
lons = scn.datasets['longitude'].data
lats = scn.datasets['latitude'].data
data = scn.datasets['1'].data

# Resampling
`SwathDefinition`, `AreaDefinition`, `GridDefinition` - classes that describe the grid geometry and cartographic projection.

* `SwathDefinition` - irregular grid
* `AreaDefinition` - regular grid in cartographic projection
* `GridDefinition` - regular grid in geographic coordinates

In [ ]:
# Resample swath to area
from pyresample import kd_tree

%timeit result = kd_tree.resample_nearest(swath_def, data, area_def, radius_of_influence=2000, epsilon=0.5)

In [ ]:
import numpy as np
from pyresample.geometry import SwathDefinition, GridDefinition, AreaDefinition

# Define swath grid
swath_def = SwathDefinition(lons=lons, lats=lats)

# Define area
area_def = AreaDefinition(area_extent=(-2056956.074, 539336.813,
                                        1097748.327,-2586425.345),
                          proj_dict=dict(proj="stere",
                                         ellps="WGS84",
                                         lat_0="90",
                                         lon_0="20",
                                         lat_ts="75",),
                           x_size=1000,
                           y_size=1000,
                           proj_id='foo',
                           area_id='bar',
                           name='foo')

# Define geographic grid
x, y = np.meshgrid(
            np.arange(-40, 20, 0.01),
            np.arange(70, 85, 0.01))

grid_def = GridDefinition(lons=x,
                          lats=y)

### Split resampling in two steps: find neighbours and retrieve resampling results


In [ ]:

%timeit valid_input_index, valid_output_index, index_array, distance_array = \
                       kd_tree.get_neighbour_info(swath_def, area_def, 2000, neighbours=1)

%timeit result = kd_tree.get_sample_from_neighbour_info('nn', area_def.shape, data, valid_input_index, valid_output_index, index_array)


### Resampling with uncertainty (weighted stddev) estimates

In [ ]:
from pyresample.utils import fwhm2sigma
result, stddev, count = kd_tree.resample_gauss(swath_def, data, area_def,
                                               radius_of_influence=1000,
                                               sigmas=fwhm2sigma(2000),
                                               fill_value=None, with_uncert=True)



### Resampling from swath to swath
KDTree supports resampling from irregular to irregular grids, therefore it's possible to resample from swath to swath

Now let's get data for the same area from [different orbit](https://lance-modis.eosdis.nasa.gov/cgi-bin/imagery/single.cgi?image=crefl1_143.A2017238130000-2017238130500.2km.jpg)

In [ ]:
# Get more data (geolocation only this time)
!wget -c ftp://ladsweb.nascom.nasa.gov/allData/61/MOD03/2017/238/MOD03.A2017238.1300.061.2017317001639.hdf


In [ ]:
scn = Scene(reader='hdfeos_l1b',
            filenames=glob.glob('./MOD03.A2017238.1300.061.2017317001639.hdf'))
scn.load(['latitude', 'longitude'])
lon_other = scn.datasets['longitude']
lat_other = scn.datasets['latitude']
swath_def_other = SwathDefinition(lons=lon_other, lats=lat_other)

In [ ]:
%timeit result = kd_tree.resample_nearest(swath_def, data, swath_def_other, radius_of_influence=2000, epsilon=0.5)